In [4]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
API_VERSION = os.getenv("API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

In [5]:
# Create Azure Model Client
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint= AZURE_OPENAI_ENDPOINT,
    api_key= AZURE_OPENAI_KEY,
    api_version= API_VERSION,
    deployment_name= "gpt-4o",  # Your deployment name
    model= AZURE_DEPLOYMENT_NAME
)

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
import json

In [ ]:
def inventory_check_tool(product: str) -> str:
    inventory_data = json.loads("\inventory.json")
    return inventory_data.get(product, "Product data is not available.")

In [ ]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks. It should break down tasks and delegate them to the appropriate agents.",
    model_client = model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        EnquiryAgent: Checks whether the enquired product is present in stock.
        PlaceOrderAgent: Takes the order if product is prsent in stock
        OrderStatusCheckAgent: Checks the order status.
        ComplaintAgent: Takes the complaint and checks regarding it.
        ResponseAgent: Its the last agent which gives output lastsly.
    
    You only plan and delegate tasks - you do not execute them yourself.
    
    When assigning tasks, use this format:
    1. <agent> : <task>
     
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)
EnquiryAgent_agent = AssistantAgent(
    "EnquiryAgent",
    description="An agent that provides current status of inventory for a given product.",
    tools=[inventory_check_tool],
    model_client = model_client,
    system_message="""
    You are a Inventory-checking agent.
    Your only tool is Inventory_check_tool - use it to fetch weatheInventory data for a given product.
    """,
)
